In [19]:

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Initialize the webdriver
driver = webdriver.Chrome()

# Visit the website
url = "https://tes.collegesource.com/publicview/TES_publicview01.aspx?rid=f080a477-bff8-46df-a5b2-25e9affdd4ed&aid=27b576bb-cd07-4e57-84d0-37475fde70ce"
driver.get(url)

# Wait for 2 seconds
import time
time.sleep(2)

# Get the page source
page_source = driver.page_source

# Parse the page source with BeautifulSoup

# Find the table element with id "gdvInstWithEQ"
table = driver.find_element(By.XPATH, '//*[@id="gdvInstWithEQ"]')

# print the table rows
# Loop through rows 0-49 to find elements with pattern gdvInstWithEQ_btnCreditFromInstName_{i}

data = {}
for i in range(50):
    try:
        # Find element by ID
        uni_elm = driver.find_element(By.ID, f"gdvInstWithEQ_btnCreditFromInstName_{i}")
        # Print the text and click the element
        uni_name = uni_elm.text
        time.sleep(1)
        uni_elm.click()
        # wait 2 seconds for the page to load
        time.sleep(5)

        
    

         
        # loop through until there is no element found
        courses = []
        counter = 0
        # Check if pagination info exists
        try:
            pagination_info = driver.find_element(By.ID, "lblCourseEQPaginationInfo")
            print(f"Pagination info found: {pagination_info.text}")
        except:
            print("No pagination info found")
        for i in range(50):
            counter += 1
            try:
                alt_course_name = driver.find_element(By.ID, f"gdvCourseEQ_lblReceiveCourseCode_{counter}")
              
                rpi_course_name = driver.find_element(By.ID, f"gdvCourseEQ_btnViewCourseEQDetail_{counter}")

                courses_data = {
                    "alt_course_name": alt_course_name.text,
                    "rpi_course_name": rpi_course_name.text
                }
                courses.append(courses_data)
                # note oth of tehse down
            except:
                break
        print(courses)
                
    except:
        # Skip if element not found 
        continue





No pagination info found
[{'alt_course_name': 'LITR 2000 LITERATURE ELECTIVE ()', 'rpi_course_name': 'ARH260 FROM DISNEY TO BURTON AND BEYOND: THE HIS OF ANIMATION (3)'}, {'alt_course_name': 'BIOL 2000 BIOLOGY ELECTIVE ()', 'rpi_course_name': 'BIO204 HUMAN ANATOMY AND PHYSIOLOGY II (4)'}, {'alt_course_name': 'BIOL 2000 BIOLOGY ELECTIVE ()', 'rpi_course_name': 'BIO208 MICROBIOLOGY (3)'}, {'alt_course_name': 'NT NOT TRANSFERABLE', 'rpi_course_name': 'CHE181 SAFETY IN LABORATORY AND STUDIO (1)'}, {'alt_course_name': 'NT NOT TRANSFERABLE', 'rpi_course_name': 'CSC156 DISCRETE STRUCTURES (3)'}, {'alt_course_name': 'NT NOT TRANSFERABLE', 'rpi_course_name': 'CSC170 INTRO TO COMPS AND THEIR APPLICATIONS (3)'}, {'alt_course_name': 'CSCI 1100 COMPUTER SCIENCE I (4)', 'rpi_course_name': 'CSC171 INTRO TO COMP PROGRAMMING (LECTURE) (4)'}, {'alt_course_name': 'NT NOT TRANSFERABLE', 'rpi_course_name': 'CSC190 COMP SCI ORIENTATION SEMINAR (1)'}, {'alt_course_name': 'ECON 1000 ECONOMICS ELECTIVE ()', 'r

In [18]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
import time

def scrape_class_data(driver, institution_name):
    print(f"\n{'='*20} Scraping courses for {institution_name} {'='*20}")
    # loop through until there is no element found
    courses = []
    # Check if pagination info exists
    # Find all elements that start with the ID pattern
    elements = driver.find_elements(By.CSS_SELECTOR, '[id^="gdvCourseEQ_lblReceiveCourseCode_"]')
    num_courses = len(elements)
    
    for i in range(num_courses):
        alt_course_name = driver.find_element(By.ID, f"gdvCourseEQ_lblReceiveCourseCode_{i}")
        rpi_course_name = driver.find_element(By.ID, f"gdvCourseEQ_btnViewCourseEQDetail_{i}")

        courses_data = {
            "alt_course_name": alt_course_name.text,
            "rpi_course_name": rpi_course_name.text
        }
        courses.append(courses_data)
    
    print(f"Found {num_courses} courses on current page")
    return courses

def scrape_uni_page(driver, institution_name):
    print(f"\n{'='*20} Processing {institution_name} {'='*20}")
    # get the first page
    all_courses = scrape_class_data(driver, institution_name)

    try:
        # number of pages
        pagination_table = driver.find_element(By.CLASS_NAME, "pagination-tes").find_element(By.TAG_NAME, "table")
        page_cells = pagination_table.find_elements(By.TAG_NAME, "td")
        num_pages = len(page_cells)
        print(f"Found {num_pages} total pages to process")

        for page in range(2, num_pages + 1):
            print(f"\n--- Processing Page {page} ---")
            # Need to refind the pagination table and link each time since page reloads
            pagination_table = driver.find_element(By.CLASS_NAME, "pagination-tes").find_element(By.TAG_NAME, "table")
            page_link = pagination_table.find_element(By.LINK_TEXT, str(page))
            page_link.click()
            time.sleep(2)
            # get the courses and extend
            curr_page_courses = scrape_class_data(driver, institution_name)
            all_courses.extend(curr_page_courses)
    except:
        # No pagination table found - just continue with single page results
        print("No pagination found - single page only")
 
    print(f"\nTotal courses found for {institution_name}: {len(all_courses)}")
    return all_courses

def navigate_to_page(driver, page_number):
    # Find the pagination table
    pagination_table = driver.find_element(By.CLASS_NAME, "pagination-tes").find_element(By.TAG_NAME, "table")
    
    if page_number > 10:
        # Need to click dots first to see pages 11-14
        dots_link = pagination_table.find_element(By.LINK_TEXT, "...")
        dots_link.click()
        time.sleep(2)
        
        # Re-find pagination table since page updated
        pagination_table = driver.find_element(By.CLASS_NAME, "pagination-tes").find_element(By.TAG_NAME, "table")
        
        # Don't need to click page number if it's 11 since dots takes us there
        if page_number > 11:
            page_link = pagination_table.find_element(By.LINK_TEXT, str(page_number))
            page_link.click()
            time.sleep(2)
    else:
        # For pages 2-10, just click the page number
        page_link = pagination_table.find_element(By.LINK_TEXT, str(page_number)) 
        page_link.click()
        time.sleep(2)

def scrape_institutions(driver, page_numbers):
    all_institutions_data = []

 
    
    for page_number in page_numbers:
        # Visit the website if first page, otherwise navigate
        if page_number == page_numbers[0]:
            url = "https://tes.collegesource.com/publicview/TES_publicview01.aspx?rid=f080a477-bff8-46df-a5b2-25e9affdd4ed&aid=27b576bb-cd07-4e57-84d0-37475fde70ce"
            driver.get(url)
            
            time.sleep(60) # in case captcha is triggered
            if page_number > 1:
                navigate_to_page(driver, page_number)
        else:
            navigate_to_page(driver, page_number)

        # Loop through buttons 0-49
        for i in range(50):
            # Find and click the button with dynamic index
            button_id = f"gdvInstWithEQ_btnCreditFromInstName_{i}"
            button = driver.find_element(By.ID, button_id)
            institution_name = button.text
            print(f"\n{'#'*80}\nProcessing institution {i+1}/50: {institution_name}\n{'#'*80}")
            try:
                button.click()
                # Wait for data to change by checking for presence of course table
                WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.ID, "gdvCourseEQ"))
                )
            except:
                print(f"Failed to click institution {institution_name} with page number {page_number}")
                time.sleep(90)
                continue
                
            # scrape the page
            courses = scrape_uni_page(driver, institution_name)
            # Store institution data
            institution_data = {
                "institution_name": institution_name,
                "courses": courses
            }
            all_institutions_data.append(institution_data)
            # click back to the first page, click btnSwitchView
            driver.find_element(By.ID, "btnSwitchView").click()
            # Wait for the institution list to be visible again
            try:
                WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.ID, "gdvInstWithEQ"))
                )
            except:
                print(f"Failed to click back to first page after {institution_name}, with page number {page_number}")
                time.sleep(90)
                continue

    print(f"\n{'='*50}\nProcessing Complete\n{'='*50}")
    print(f"Total institutions processed: {len(all_institutions_data)}")
    for inst in all_institutions_data:
        print(f"\nInstitution: {inst['institution_name']}")
        print(f"Number of courses: {len(inst['courses'])}")
        
    return all_institutions_data

import threading
all_data = []
lock = threading.Lock()

def process_pages(page_range):
    driver = webdriver.Chrome(options=webdriver.ChromeOptions())
    try:
        data = scrape_institutions(driver, page_range)
        if data:
            with lock:
                all_data.extend(data)
    except Exception as e:
        print(f"Error processing pages {page_range}: {str(e)}")
    finally:
        driver.quit()

# Define the page ranges for each thread
page_ranges = [
    [1, 11]
]

threads = []
for page_range in page_ranges:
    thread = threading.Thread(target=process_pages, args=(page_range,))
    threads.append(thread)
    thread.start()
    print(f"Started thread for pages {page_range}")

# Wait for all threads to complete
for thread in threads:
    thread.join()

print(f"Total institutions collected: {len(all_data)}")


Started thread for pages [1, 2, 3, 4]
Started thread for pages [5, 6, 7, 8]
Started thread for pages [9, 10, 11, 12]
Started thread for pages [13, 14]

################################################################################
Processing institution 1/50: ADELPHI UNIVERSITY
################################################################################

################################################################################
Processing institution 1/50: GREAT BAY COMMUNITY COLLEGE
################################################################################

################################################################################
Processing institution 1/50: PENNSYLVANIA STATE UNIVERSITY-ERIE BEHREND COLLEGE
################################################################################

==================== Processing ADELPHI UNIVERSITY ====================

==================== Scraping courses for ADELPHI UNIVERSITY ====================
Found 18 courses on

In [34]:
all_data = []
import json
with open('all_data_clean.json', 'r') as f:
    all_data = json.load(f)


In [35]:
# rerun a specific page
page_range = [8, 8]
data = scrape_institutions(driver, page_range)
all_data.extend(data)

MaxRetryError: HTTPConnectionPool(host='localhost', port=53276): Max retries exceeded with url: /session/a7db2eef73fbcb828a9bd43f78559aae/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x115defb50>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [37]:



# Remove duplicates based on institution name and check for duplicate data
seen_institutions = {}
unique_data = []
duplicate_data = []

for institution in all_data:
    inst_name = institution['institution_name']
    if inst_name not in seen_institutions:
        seen_institutions[inst_name] = institution
        unique_data.append(institution)
    else:
        # Check if the duplicate has identical course data
        existing_inst = seen_institutions[inst_name]
        if (existing_inst['courses'] == institution['courses'] and 
            all(course1 == course2 for course1, course2 in zip(existing_inst['courses'], institution['courses']))):
            print(f"Found identical duplicate for: {inst_name}")
        else:
            print(f"Found different duplicate for: {inst_name}")
            print(f"Original courses: {len(existing_inst['courses'])}")
            print(f"Duplicate courses: {len(institution['courses'])}")
        duplicate_data.append(institution)

print(f"Original number of institutions: {len(all_data)}")
print(f"Number of institutions after removing duplicates: {len(unique_data)}")
print(f"Number of duplicates found: {len(duplicate_data)}")

# Replace all_data with deduplicated data
all_data = unique_data


Original number of institutions: 388
Number of institutions after removing duplicates: 388
Number of duplicates found: 0


In [28]:
# save to json
import json
with open('all_data_clean.json', 'w') as f:
    json.dump(all_data, f)


In [41]:
!pip3 install pandas

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 11.3 MB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 5.3 MB 25.5 MB/s eta 0:00:01
     |████████████████████████████████| 508 kB 95.6 MB/s eta 0:00:01
     |████████████████████████████████| 346 kB 89.0 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [42]:
import pandas as pd
# Load the cleaned data
with open('all_data_clean.json', 'r') as f:
    all_data = json.load(f)

    # Convert the data to a pandas DataFrame for easier spreadsheet export
    import pandas as pd
    
    # Flatten the nested data structure
    flattened_data = []
    for institution in all_data:
        inst_name = institution['institution_name']
        for course in institution['courses']:
            course_data = {
                'Institution': inst_name,
                'Alt Course': course['rpi_course_name'],  # Swapped these two keys
                'RPI Course': course['alt_course_name']   # Swapped these two keys
            }
            flattened_data.append(course_data)
    
    # Create DataFrame
    df = pd.DataFrame(flattened_data)
    
    # Export to Excel
    df.to_excel('course_data.xlsx', index=False)
    #csv as well
    df.to_csv('course_data.csv', index=False)

    print(f"Data exported to course_data.xlsx with {len(df)} courses from {len(all_data)} institutions")


ValueError: No engine for filetype: 'csv'